In [2]:
import IMAS
import FuseExchangeProtocol as FXP
import IJulia

if true
    # if using Redis via docker
    REDIS_HOST = "localhost"
    REDIS_PORT = 55000
    REDIS_PASSWORD = "redispw"
else
    REDIS_HOST = "redis-19689.c281.us-east-1-2.ec2.cloud.redislabs.com"
    REDIS_PORT = 19689
    REDIS_PASSWORD = ""
end;

In [3]:
function service_ip_control(client::FXP.Client, session_id::String, service_name::String; timeout=10.0)
    controller = IMAS.controllers__linear_controller()
    controller.name = "ip"
    
    while true
        # pop inputs
        inputs = FXP.json_pop(client, session_id, service_name, :service; timeout, error_on_timeout=false)
        if inputs === nothing
            println("DONE: $(session_id) $(service_name)")
            break
        end
        inputs = NamedTuple(inputs)

        # initialize controller with given PID coefficients
        if ismissing(controller.outputs, :data)
            IMAS.pid_controller(controller, inputs.P, inputs.I, inputs.D)
        end
        # run controller
        control = controller(inputs.setpoint, inputs.value, inputs.time)
        
        # push output
        FXP.json_push(client, session_id, service_name, :service; control)
    end
end

service_ip_control (generic function with 1 method)

In [4]:
# automated spawn
subscription_client = FXP.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
FXP.register_service(subscription_client, "ip_control", service_ip_control)

Jedis.Client("localhost", 55000, 0, "redispw", "", Sockets.TCPSocket(RawFD(55) active, 0 bytes waiting), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (2, 5926473904, 5945552272)), nothing, true, Set(["ip_control"]), Set{String}(), true, 1, Jedis.var"#13#15"(), false, 60)

AssertionError("whoami ∈ [:provider, :requestor]")


In [5]:
fieldnames(typeof(subscription_client))

(:host, :port, :database, :password, :username, :socket, :lock, :ssl_config, :is_subscribed, :subscriptions, :psubscriptions, :retry_when_closed, :retry_max_attempts, :retry_backoff, :keepalive_enable, :keepalive_delay)

In [ ]:
# manual spawn
subscription_client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
FXP.register_service(subscription_client, "ip_control", x->println(x))

In [ ]:
service_ip_control("8136311451093097212", "ip_control")